In [1]:
from aiida import orm, load_profile
from aiida_aurora.calculations import BatteryCyclerExperiment

load_profile()

Profile<uuid='14f418767fa44f6f9d65ac40c9d45d39' name='default'>

In [2]:
qb = (
    orm.QueryBuilder()
    .append(
        orm.Group,
        filters={
            "label": "aurora/experiments/ga-231012-4.2",
        },
        tag="group",
    )
    .append(
        BatteryCyclerExperiment,
        with_group="group",
        tag="experiment",
    )
    .append(
        orm.ArrayData,
        with_incoming="experiment",
    )
)

In [3]:
import numpy as np

t, I, V = [], [], []  # noqa: E741
result: orm.ArrayData = qb.first(flat=True)

time = result.get_array("step0_uts")
time -= time[0]
current = result.get_array("step0_I_n")
voltage = result.get_array("step0_Ewe_n")

t = np.array(time)
I = np.array(current)
V = np.array(voltage)

In [4]:
import pandas as pd

df = pd.DataFrame({"t": t, "I": I, "Ewe": V}).apply(lambda col: col.explode())
# df

In [5]:
df.to_csv("data/test.csv", index=False)

In [6]:
df["t"] = df["t"].apply(lambda row: row / 3600)
df["I"] = df["I"].apply(lambda row: row * 1000)

In [7]:
df.to_json("data/test.json", orient="split", index=False)